### Intensité et classe de signal des lésions (01 -> 13)

In [ ]:
from pathlib import Path
import csv
import SimpleITK as sitk

data_path = data_path = Path.cwd().parent.parent / "data"

signal_class_dir = data_path / "13_lesion_signal_class_with_intensity"
if not signal_class_dir.exists():
    signal_class_dir.mkdir(parents = True, exist_ok = True)

In [6]:
def classify(lesion_mean, ref_mean, ref_std, factor=1.5):
    """
    Classification basée sur le Z-score.
    """
    upper_bound = ref_mean + (factor * ref_std)
    lower_bound = ref_mean - (factor * ref_std)
    
    if lesion_mean > upper_bound:
        return "hypersignal"
    elif lesion_mean < lower_bound:
        return "hyposignal"
    else:
        return "isosignal"

In [ ]:
"""
01 -> 13
"""

mslesseg_path = data_path / "01_MSLesSeg_Dataset"

for mask_path in mslesseg_path.rglob("*_MASK.nii.gz"):

    t1_path = list(mask_path.parent.glob("*_T1.nii.gz"))[0]
    t2_path = list(mask_path.parent.glob("*_T2.nii.gz"))[0]
    flair_path = list(mask_path.parent.glob("*_FLAIR.nii.gz"))[0]

    arg = mask_path.stem.split("_")

    if len(arg) == 3:
        patient_id, timepoint = arg[0], arg[1]
    elif len(arg) == 2:
        patient_id, timepoint = arg[0], "T1"


    # Lecture images
    mask_img_orig  = sitk.ReadImage(str(mask_path))
    t1_img_orig    = sitk.ReadImage(str(t1_path))
    t2_img_orig    = sitk.ReadImage(str(t2_path))
    flair_img_orig = sitk.ReadImage(str(flair_path))


    # Traitement
    lesion_bin = mask_img_orig > 0
    lesion_cc = sitk.ConnectedComponent(lesion_bin)

    brain_bin = t1_img_orig > 0
    ref_bin   = sitk.And(brain_bin, sitk.Not(lesion_bin))
    ref_label = sitk.Cast(ref_bin, sitk.sitkUInt8)


    modalities = {
        "T1": sitk.Cast(t1_img_orig, sitk.sitkFloat64),
        "T2": sitk.Cast(t2_img_orig, sitk.sitkFloat64),
        "FLAIR": sitk.Cast(flair_img_orig, sitk.sitkFloat64)
    }


    # =============================
    # Un CSV par modalité
    # =============================
    for mod, img in modalities.items():

        # Nom du fichier
        csv_name = f"{patient_id}_{timepoint}_{mod}_signal_class.csv"
        csv_path = signal_class_dir / csv_name


        with open(csv_path, "w", newline="") as f:

            writer = csv.writer(f)

            # Header
            writer.writerow([
                "patient",
                "timepoint",
                "modality",
                "lesion_id",
                "lesion_mean",
                "lesion_min",
                "lesion_max",
                "ref_mean",
                "ref_std",
                "signal_class"
            ])


            # Stats référence
            stats_ref = sitk.LabelStatisticsImageFilter()
            stats_ref.Execute(img, ref_label)

            if not stats_ref.HasLabel(1):
                continue


            ref_mean = stats_ref.GetMean(1)
            ref_std  = stats_ref.GetSigma(1)


            # Stats lésions
            stats_lesions = sitk.LabelStatisticsImageFilter()
            stats_lesions.Execute(img, lesion_cc)


            for lesion_id in sorted(stats_lesions.GetLabels()):

                if lesion_id == 0:
                    continue

                lesion_mean = stats_lesions.GetMean(lesion_id)
                lesion_min  = stats_lesions.GetMinimum(lesion_id)
                lesion_max  = stats_lesions.GetMaximum(lesion_id)

                signal_class = classify(lesion_mean, ref_mean, ref_std)

                writer.writerow([
                    patient_id,
                    timepoint,
                    mod,
                    int(lesion_id),
                    round(float(lesion_mean), 4),
                    round(float(lesion_min), 4),
                    round(float(lesion_max), 4),
                    round(float(ref_mean), 4),
                    round(float(ref_std), 4),
                    signal_class
                ])